In [ ]:
# perform a SVM classification on the data
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_recall_curve, auc, roc_curve, confusion_matrix



In [ ]:
def output(y_test, y_pred, y_pred_proba):
    # Classification report and accuracy
    print(classification_report(y_test, y_pred))
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy )

    # Calculate AUROC
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print("AUROC:", roc_auc)

    # Calculate AUPRC
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    auprc = auc(recall, precision)
    print("AUPRC:", auprc)

    # Calculate sensitivity and specificity
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

    # Plot ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

    # Plot Precision-Recall curve
    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label='PR curve (area = %0.2f)' % auprc)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()
    wandb.log({"Accucary:": accuracy, "AUROC": roc_auc, "AUPRC": auprc, "senstivity": sensitivity, "specificity": specificity})



In [ ]:
data_path1 = "/home/vera/Documents/AI/AILS-MICCAI-UWF4DR-Challenge/data/raw/UWF4DRChallengeData/Task 1 Image Quality Assessment/1. Images/1. Training"
data_path2 = "/home/vera/Documents/AI/AILS-MICCAI-UWF4DR-Challenge/data/raw/UWF4DRChallengeData/Task 2 Referable DR and Task 3 DME/1. Images/1. Training"
data1 = load_images(data_path1)
data2 = load_images(data_path2)
data = np.concatenate((data1, data2), axis=0)
label_path = "/home/vera/Documents/AI/AILS-MICCAI-UWF4DR-Challenge/data/raw/UWF4DRChallengeData/Task 1 Image Quality Assessment/2. Groundtruths/1. Training.csv"
# Create a binary classification task with an SVM
y = np.loadtxt(label_path, delimiter=',', skiprows=1, usecols=1)

In [ ]:
n_samples, h, w, c = data.shape
flattened_images = data.reshape(n_samples, -1)

# Standardize the data
scaler = StandardScaler()
flattened_images_scaled = scaler.fit_transform(flattened_images)

In [ ]:
random_state = 42
kernel = "linear"
C = 1.0
test_size = 0.2

wandb.init(project="UWF4DR_Challenge", entity="pilsvera", name = "SVC")
wandb.config = {"model": "SVM", "kernel": kernel, "C": C, "random_state": random_state, "test_size": test_size}


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(flattened_images_scaled, y, test_size=test_size, random_state=random_state)

In [ ]:
svm_model = SVC(kernel=kernel, probability=True, random_state=random_state)
#model = SVC(kernel=kernel, class_weight={0: 1, 1: 10}, probability=True, random_state=random_state)

svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]
output(y_test, y_pred, y_pred_proba)